In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df

In [3]:
num_cars = 100
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data

data.head()

car_id,timestamp,latitude,longitude,notification
str,str,f64,f64,str
"""car_0""","""2023-01-01T00:…",26.365421,-96.060872,null
"""car_0""","""2023-01-01T00:…",26.365421,-96.060872,null
"""car_0""","""2023-01-01T00:…",26.365421,-96.060872,"""engine_check"""
"""car_0""","""2023-01-01T00:…",26.365421,-96.060872,"""engine_check"""
"""car_0""","""2023-01-01T00:…",26.365421,-96.060872,"""engine_check"""


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [4]:
# Convertir el campo "timestamp" a formato de tiempo en Polars
data = data.with_columns(pl.col("timestamp").str.to_datetime())

# Verificar la conversión
data.head()


car_id,timestamp,latitude,longitude,notification
str,datetime[μs],f64,f64,str
"""car_0""",2023-01-01 00:00:00,26.365421,-96.060872,null
"""car_0""",2023-01-01 00:12:38.307630,26.365421,-96.060872,null
"""car_0""",2023-01-01 00:18:24.826119,26.365421,-96.060872,"""engine_check"""
"""car_0""",2023-01-01 00:14:15.447030,26.365421,-96.060872,"""engine_check"""
"""car_0""",2023-01-01 00:06:29.547796,26.365421,-96.060872,"""engine_check"""


In [5]:
data

car_id,timestamp,latitude,longitude,notification
str,datetime[μs],f64,f64,str
"""car_0""",2023-01-01 00:00:00,26.365421,-96.060872,null
"""car_0""",2023-01-01 00:12:38.307630,26.365421,-96.060872,null
"""car_0""",2023-01-01 00:18:24.826119,26.365421,-96.060872,"""engine_check"""
"""car_0""",2023-01-01 00:14:15.447030,26.365421,-96.060872,"""engine_check"""
"""car_0""",2023-01-01 00:06:29.547796,26.365421,-96.060872,"""engine_check"""
…,…,…,…,…
"""car_99""",2023-01-07 23:16:46.596648,20.186539,-92.927617,"""tire_pressure"""
"""car_99""",2023-01-07 23:25:23.811186,20.186539,-92.927617,"""tire_pressure"""
"""car_99""",2023-01-07 23:44:30.012544,20.185603,-92.92265,"""engine_check"""


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es necesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [6]:
# Ordenar los registros por car_id y timestamp
data = data.sort(["car_id", "timestamp"])

data = data.with_columns(
    pl.col("timestamp")
    .diff()
    .over("car_id")
    .alias("notification_time")
)


In [7]:
data[5840:5850]

car_id,timestamp,latitude,longitude,notification,notification_time
str,datetime[μs],f64,f64,str,duration[μs]
"""car_0""",2023-01-07 20:58:02.906700,26.581662,-96.033041,"""engine_check""",6s 668364µs
"""car_0""",2023-01-07 20:59:14.287193,26.581662,-96.033041,"""engine_check""",1m 11s 380493µs
"""car_0""",2023-01-07 20:59:22.206289,26.581662,-96.033041,"""engine_check""",7s 919096µs
"""car_0""",2023-01-07 21:04:18.434912,26.581662,-96.033041,"""tire_pressure""",4m 56s 228623µs
"""car_0""",2023-01-07 21:08:09.963318,26.587337,-96.027698,"""low_fuel""",3m 51s 528406µs
"""car_0""",2023-01-07 21:08:11.810184,26.587337,-96.027698,"""low_fuel""",1s 846866µs
"""car_0""",2023-01-07 21:10:34.162185,26.587337,-96.027698,null,2m 22s 352001µs
"""car_0""",2023-01-07 21:11:20.193687,26.587337,-96.027698,"""tire_pressure""",46s 31502µs
"""car_0""",2023-01-07 21:13:35.498107,26.587337,-96.027698,"""tire_pressure""",2m 15s 304420µs


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [8]:
import math

#Se usará la fórmula de haversine para calcular la distancia en kilómetros entre dos puntos geográficos


def haversine_distance(lat1, lon1, lat2, lon2):
    
    R = 6371  # Radio de la Tierra en kilómetros

    if lat1 is None or lon1 is None or lat2 is None or lon2 is None:
        return None

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c



In [9]:
# Crear la variable distance
data = data.with_columns([
    pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),
    pl.col("longitude").shift(1).over("car_id").alias("prev_longitude"), 
])

nueva = data.map_rows(lambda t: haversine_distance(t[2], t[3], t[6], t[7]))



In [10]:
type(nueva)

polars.dataframe.frame.DataFrame

In [11]:
data=data.with_columns(pl.Series(name="distance", values=nueva)) 


In [12]:
data[5840:5850]


car_id,timestamp,latitude,longitude,notification,notification_time,prev_latitude,prev_longitude,distance
str,datetime[μs],f64,f64,str,duration[μs],f64,f64,f64
"""car_0""",2023-01-07 20:58:02.906700,26.581662,-96.033041,"""engine_check""",6s 668364µs,26.581662,-96.033041,0.0
"""car_0""",2023-01-07 20:59:14.287193,26.581662,-96.033041,"""engine_check""",1m 11s 380493µs,26.581662,-96.033041,0.0
"""car_0""",2023-01-07 20:59:22.206289,26.581662,-96.033041,"""engine_check""",7s 919096µs,26.581662,-96.033041,0.0
"""car_0""",2023-01-07 21:04:18.434912,26.581662,-96.033041,"""tire_pressure""",4m 56s 228623µs,26.581662,-96.033041,0.0
"""car_0""",2023-01-07 21:08:09.963318,26.587337,-96.027698,"""low_fuel""",3m 51s 528406µs,26.581662,-96.033041,0.824931
"""car_0""",2023-01-07 21:08:11.810184,26.587337,-96.027698,"""low_fuel""",1s 846866µs,26.587337,-96.027698,0.0
"""car_0""",2023-01-07 21:10:34.162185,26.587337,-96.027698,null,2m 22s 352001µs,26.587337,-96.027698,0.0
"""car_0""",2023-01-07 21:11:20.193687,26.587337,-96.027698,"""tire_pressure""",46s 31502µs,26.587337,-96.027698,0.0
"""car_0""",2023-01-07 21:13:35.498107,26.587337,-96.027698,"""tire_pressure""",2m 15s 304420µs,26.587337,-96.027698,0.0


In [13]:
data.dtypes

[String,
 Datetime(time_unit='us', time_zone=None),
 Float64,
 Float64,
 String,
 Duration(time_unit='us'),
 Float64,
 Float64,
 Float64]

In [14]:
data.columns

['car_id',
 'timestamp',
 'latitude',
 'longitude',
 'notification',
 'notification_time',
 'prev_latitude',
 'prev_longitude',
 'distance']

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [15]:
print(data.dtypes[1])

Datetime(time_unit='us', time_zone=None)


Profe, lo intenté hcer con group_by_dynamic y de verdad no me salía. Claude me sugirió que lo hiciera con un group_by normal, y creo que el resultado que me dió es bueno, sin embargo, tuve que ir creando los intervalos a mano. Sé que no es lo más eficiente, pero al menos encontré una solución que funcionó, por lo que espero lo considere

In [33]:
x="5m"

data = data.with_columns(
    pl.col("timestamp").dt.truncate(x).alias("intervalo_manual")
)

data2 = data.groupby(["car_id", "intervalo_manual"])




/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/3031122534.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  data2 = data.groupby(["car_id", "intervalo_manual"])


In [34]:
#3.1
stats = data2.agg([
    pl.col("notification_time").dt.seconds().mean().alias("mean"),
    pl.col("notification_time").dt.seconds().median().alias("median"),
    pl.col("notification_time").dt.seconds().var().alias("variance"),
    pl.col("notification_time").dt.seconds().max().alias("max"),
    pl.col("notification_time").dt.seconds().min().alias("min"),
])

stats=stats.sort(["car_id", "intervalo_manual"])
stats

/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/4096314768.py:3: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().mean().alias("mean"),
/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/4096314768.py:4: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().median().alias("median"),
/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/4096314768.py:5: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().var().alias("variance"),
/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/4096314768.py:6: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().max().alias("max"),
/var/folders/20/32nt1tgd7dn17l8pghpkyhhc0000gn/T/ipykernel_3528/4096314

car_id,intervalo_manual,mean,median,variance,max,min
str,datetime[μs],f64,f64,f64,i64,i64
"""car_0""",2023-01-01 00:00:00,46.5,46.5,2520.5,82,11
"""car_0""",2023-01-01 00:05:00,147.666667,148.0,4160.333333,212,83
"""car_0""",2023-01-01 00:10:00,52.833333,53.5,872.566667,87,2
"""car_0""",2023-01-01 00:15:00,246.0,246.0,null,246,246
"""car_0""",2023-01-01 00:30:00,177.0,57.0,85378.5,695,6
…,…,…,…,…,…,…
"""car_99""",2023-01-07 23:10:00,1068.0,1068.0,null,1068,1068
"""car_99""",2023-01-07 23:15:00,136.0,136.0,null,136,136
"""car_99""",2023-01-07 23:25:00,517.0,517.0,null,517,517


In [36]:
#3.2
stats2 = data2.agg([
    pl.col("distance").mean().alias("mean_distance"),
    pl.col("distance").median().alias("median_distance"),
    pl.col("distance").var().alias("variance_distance"),
    pl.col("distance").max().alias("max_distance"),
    pl.col("distance").min().alias("min_distance"),
])

stats2 = stats2.sort(["car_id", "intervalo_manual"])
stats2


car_id,intervalo_manual,mean_distance,median_distance,variance_distance,max_distance,min_distance
str,datetime[μs],f64,f64,f64,f64,f64
"""car_0""",2023-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 00:05:00,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 00:15:00,0.0,0.0,null,0.0,0.0
"""car_0""",2023-01-01 00:30:00,0.195676,0.0,0.191446,0.978382,0.0
…,…,…,…,…,…,…
"""car_99""",2023-01-07 23:10:00,0.758479,0.758479,null,0.758479,0.758479
"""car_99""",2023-01-07 23:15:00,0.0,0.0,null,0.0,0.0
"""car_99""",2023-01-07 23:25:00,0.0,0.0,null,0.0,0.0
